In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.insert(0, '../')
from gantools.data import transformation
from gantools.regressor import Regressor, get_regressor_outputs
from gantools.gansystem import NNSystem
from gantools import plot
from gantools import utils

from cosmotools.data import load
from cosmotools.utils import histogram_large, find_minmax_large


In [ ]:
# Note: some of the parameters don't make sense for the fake dataset
ns = 128 # Resolution of the image
try_resume = True # Try to resume previous simulation

# Data handling

Load the data

In [ ]:
dataset = load.load_params_dataset(filename='kids_reg_train.h5', batch=10000, transform=transformation.random_transpose_2d, shape=[ns, ns])

In [ ]:
dataset.N

In [ ]:
vmin, vmax = find_minmax_large(dataset)

In [ ]:
histo, x = histogram_large(dataset, lim=(vmin, vmax))

In [ ]:
plot.plot_histogram(x, histo)
print('min: {}'.format(vmin))
print('max: {}'.format(vmax))

Let us plot 16 images

In [ ]:
vmax = 0.3 # Set arbitrary vmax to have nicer plot

In [ ]:
def title_func(params):
    return "$\Omega_M$: " + str(params[0])[0:7] + ", $\sigma_8$: " + str(params[1])[0:7]

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(15,15))
idx = 0
imgs = dataset.get_samples(N=16)[0]
params = dataset.get_samples(N=16)[1]
for row in ax:
    for col in row:
        col.imshow(imgs[idx, :, :, 0], vmin=vmin, vmax=vmax)
        col.axis('off')
        col.set_title(title_func(params[idx]))
        idx = idx + 1
fig.tight_layout()

# Define parameters for the WGAN

In [ ]:
time_str = '2D_mac'
global_path = '../saved_results/Regressor/'

name = 'Kids_Regressor_' + str(ns) + '_smart_' + time_str

## Parameters

In [ ]:
bn = False

# Parameters for the regressor
params_regressor = dict()
params_regressor['full'] = [512, 256, 128]
params_regressor['nfilter'] = [32, 64, 128, 256, 512]
params_regressor['batch_norm'] = [bn, bn, bn, bn, bn]
params_regressor['shape'] = [[7, 7], [5, 5], [5, 5], [5,5], [3,3]]
params_regressor['stride'] = [1, 2, 2, 2, 2]

# Optimization parameters
params_optimization = dict()
params_optimization['learning_rate'] = 3e-5
params_optimization['batch_size'] = 64
params_optimization['epoch'] = 100

# all parameters
params = dict()
params['net'] = dict() # All the parameters for the model
params['net']['regressor'] = params_regressor
params['net']['shape'] = [ns, ns, 1] # Shape of the image
params['net']['cond_params'] = 2
params['optimization'] = params_optimization
params['summary_every'] = 2000 # Tensorboard summaries every ** iterations
params['print_every'] = 1000 # Console summaries every ** iterations
params['save_every'] = 10000 # Save the model every ** iterations
params['summary_dir'] = os.path.join(global_path, name +'_summary/')
params['save_dir'] = os.path.join(global_path, name + '_checkpoints/')


In [ ]:
resume, params = utils.test_resume(try_resume, params)
params['summary_dir'] = os.path.join(global_path, name +'_summary/')
params['save_dir'] = os.path.join(global_path, name + '_checkpoints/')

# Build the model

In [ ]:
reg = NNSystem(Regressor, params)

# Train the model

In [ ]:
reg.train(dataset, resume=resume)

# Evaluation

In [ ]:
checkpoint = 140000
checkpoint = None

In [ ]:
N = 2000
batch_size = 250

In [ ]:
# Create bacth
dat = load.load_params_dataset(filename='kids_reg_test.h5', batch=N, shape=[ns, ns])
images, y_real = dat.get_samples(N=N)
y_pred, _ = get_regressor_outputs(reg, images, y_real, checkpoint, batch_size)

In [ ]:
print(y_real[:5])
print(y_pred[:5, :2])
print(mean_squared_error(y_real, y_pred[:, :2]))

For the whole test set

In [ ]:
dat = load.load_params_dataset(filename='kids_reg_test.h5', batch=batch_size, shape=[ns, ns])
images = []
y_real = []
err = []
idx = 0
for b in dat.iter():
    images.append(b[0][0])
    y_real.append(b[0][1])
    idx = idx + 1
    if idx % batch_size == 0:
        images = np.array(images)
        y_real = np.vstack(y_real)
        y_pred, _ = get_regressor_outputs(reg, images, y_real, checkpoint, batch_size)
        err.append(mean_squared_error(y_real, y_pred))
        images = []
        y_real = []
err = np.array(err)

In [ ]:
print(err.mean())